In [1]:
import os
from nipype.interfaces import io as nio  # Data i/o
from nipype.interfaces import spm as spm  # spm
# from nipype.interfaces import matlab as mlab    # how to run matlab
from nipype.interfaces import fsl as fsl  # fsl
from nipype.interfaces import utility as niu  # utility
from nipype.pipeline import engine as pe  # pypeline engine
from nipype.algorithms import rapidart as ra  # artifact detection
from nipype.algorithms import modelgen as model  # model specification
from nipype.algorithms import misc
from nipype.interfaces.utility import Rename

In [6]:
data_dir = '/home/chan/data/flanker/'
subject_list = ["08"]
runs = ["1", "2"]
infosource = pe.Node(interface=niu.IdentityInterface(fields=['subject_id', 'run']), name='infosource')
infosource.iterables = [('subject_id', subject_list), ('run', runs)]

In [7]:
datasource = pe.Node(
    interface=nio.DataGrabber(
        infields=['subject_id', 'run'], outfields=['func', 'anat']),
    name='datasource')
datasource.inputs.base_directory = data_dir
datasource.inputs.template = '*'
datasource.inputs.sort_filelist = True

datasource.inputs.template_args = {
    'anat': [['subject_id', 'subject_id']],
    'func': [['subject_id', 'subject_id', 'run']]
}

datasource.inputs.field_template = {'anat': 'sub-%s/anat/sub-%s_T1w.nii.gz',
                            'func': 'sub-%s/func/sub-%s_task-flanker_run-%s_bold.nii.gz'}

In [8]:
gzFunc = pe.Node(misc.Gunzip(), name='gzFunc')
gzAnat = pe.Node(misc.Gunzip(), name='gzAnat')

In [9]:
realign = pe.Node(interface=spm.Realign(), name="realign")
realign.inputs.register_to_mean = True

stty: 'standard input': Inappropriate ioctl for device


In [10]:
# input: in_files
# output: timecorrected_files
timeslice = pe.Node(interface=spm.SliceTiming(), name='timeslicing')
# timeslice.inputs.in_files = 'functional.nii'
timeslice.inputs.num_slices = 40
timeslice.inputs.time_repetition = 2
# TA = TR - (TR/NUM_SLICES)
timeslice.inputs.time_acquisition = 2.-(2./40)
timeslice.inputs.slice_order = [i for i in range(1, 40, 2)] + [i for i in range(2, 41, 2)]
timeslice.inputs.ref_slice = 1

In [11]:
# input: 
#   source: should be anatomical file
#   target: should be mean func file processed after realignment
# output: 
#   coregistered_files
#   coregistered_source

coregister = pe.Node(interface=spm.Coregister(), name="coregister")
# should be estimate and reslice
coregister.inputs.jobtype = 'estwrite'

In [13]:
# input: channel_files
# output: bias_corrected_images, forward_deformation_field
segment = pe.Node(interface=spm.NewSegment(), name='segment')
# [Inverse, Forward]
segment.inputs.write_deformation_fields = [False, True]
# prob map, gaussian, native tissue, warped tissue
# ((tissue probability map, frame num), Num gaussians, (Native space, dartel imported), (Unmodulated, Modulated))
tissue1 = (('/home/chan/tools/spm12/tpm/TPM.nii', 1), 1, (True, False), (False, False))
tissue2 = (('/home/chan/tools/spm12/tpm/TPM.nii', 2), 1, (True, False), (False, False))
tissue3 = (('/home/chan/tools/spm12/tpm/TPM.nii', 3), 2, (True, False), (False, False))
tissue4 = (('/home/chan/tools/spm12/tpm/TPM.nii', 4), 3, (True, False), (False, False))
tissue5 = (('/home/chan/tools/spm12/tpm/TPM.nii', 5), 4, (True, False), (False, False))
tissue6 = (('/home/chan/tools/spm12/tpm/TPM.nii', 6), 2, (False, False), (False, False))
segment.inputs.tissues = [tissue1, tissue2, tissue3, tissue4, tissue5, tissue6]

In [26]:
# input: 
#   deformation_file: from segment deformation
#   apply_to_files
# output:
#   normalized_files
#   normalized_image
normalize = pe.Node(interface=spm.Normalize12(), name="normalize")
# normalize = interface=spm.Normalize12()
normalize.inputs.jobtype = 'write'

In [20]:
# test
# normalize = interface=spm.Normalize12()
# normalize.inputs.jobtype = 'write'
# normalize.inputs.deformation_file = '/home/chan/spm_test/anat/y_rsub-08_T1w.nii'
# normalize.inputs.apply_to_files = '/home/chan/spm_test/func/arsub-08_task-flanker_run-1_bold.nii'
# normalize.run()

stty: 'standard input': Inappropriate ioctl for device


In [21]:
smooth = pe.Node(interface=spm.Smooth(), name="smooth")
fwhmlist = [4]
smooth.iterables = ('fwhm', fwhmlist)

In [22]:
datasink = pe.Node(nio.DataSink(), name='sinker')
datasink.inputs.base_directory = os.path.join(os.getcwd(), 'processed')

In [30]:
workflow = pe.Workflow(name='datagrab', base_dir='tmp')
workflow.connect([
    (infosource, datasource, [('subject_id', 'subject_id'), ('run', 'run')]),
    (datasource, gzFunc, [('func', 'in_file')]),
    (datasource, gzAnat, [('anat', 'in_file')]),

    (gzFunc, realign, [('out_file', 'in_files')]),
    (realign, timeslice, [('realigned_files', 'in_files')]),

    (realign, coregister, [('mean_image', 'target')]),
    (gzAnat, coregister, [('out_file', 'source')]),

    (coregister, segment, [('coregistered_source', 'channel_files')]),

    (segment, normalize, [('forward_deformation_field', 'deformation_file')]),
    (timeslice, normalize, [('timecorrected_files', 'apply_to_files')]),

    (normalize, smooth, [('normalized_files', 'in_files')]),

    (smooth, datasink, [('smoothed_files', 'smoothed')]),
])
results = workflow.run()

230101-16:45:42,820 nipype.workflow INFO:
	 Workflow datagrab settings: ['check', 'execution', 'logging', 'monitoring']
230101-16:45:42,845 nipype.workflow INFO:
	 Running serially.
230101-16:45:42,846 nipype.workflow INFO:
	 [Node] Setting-up "datagrab.datasource" in "/home/chan/Documents/graduate-essay/tmp/datagrab/_run_1_subject_id_08/datasource".
230101-16:45:42,849 nipype.workflow INFO:
	 [Node] Executing "datasource" <nipype.interfaces.io.DataGrabber>
230101-16:45:42,850 nipype.workflow INFO:
	 [Node] Finished "datasource", elapsed time 0.000453s.
230101-16:45:42,852 nipype.workflow INFO:
	 [Node] Setting-up "datagrab.datasource" in "/home/chan/Documents/graduate-essay/tmp/datagrab/_run_2_subject_id_08/datasource".
230101-16:45:42,855 nipype.workflow INFO:
	 [Node] Executing "datasource" <nipype.interfaces.io.DataGrabber>
230101-16:45:42,856 nipype.workflow INFO:
	 [Node] Finished "datasource", elapsed time 0.000275s.
230101-16:45:42,858 nipype.workflow INFO:
	 [Node] Setting-up 

stty: 'standard input': Inappropriate ioctl for device


230101-16:46:30,562 nipype.workflow INFO:
	 [Node] Finished "coregister", elapsed time 22.841995s.
230101-16:46:30,566 nipype.workflow INFO:
	 [Node] Setting-up "datagrab.segment" in "/home/chan/Documents/graduate-essay/tmp/datagrab/_run_1_subject_id_08/segment".
230101-16:46:30,568 nipype.workflow INFO:
	 [Node] Outdated cache found for "datagrab.segment".
230101-16:46:30,573 nipype.workflow INFO:
	 [Node] Executing "segment" <nipype.interfaces.spm.preprocess.NewSegment>


stty: 'standard input': Inappropriate ioctl for device


230101-16:47:16,343 nipype.workflow INFO:
	 [Node] Finished "segment", elapsed time 45.769258s.
230101-16:47:16,347 nipype.workflow INFO:
	 [Node] Setting-up "datagrab.segment" in "/home/chan/Documents/graduate-essay/tmp/datagrab/_run_2_subject_id_08/segment".
230101-16:47:16,349 nipype.workflow INFO:
	 [Node] Outdated cache found for "datagrab.segment".
230101-16:47:16,354 nipype.workflow INFO:
	 [Node] Executing "segment" <nipype.interfaces.spm.preprocess.NewSegment>


...

stty: 'standard input': Inappropriate ioctl for device


230101-16:49:38,770 nipype.workflow INFO:
	 [Node] Finished "smooth", elapsed time 26.548919s.
230101-16:49:38,772 nipype.workflow INFO:
	 [Node] Setting-up "datagrab.smooth" in "/home/chan/Documents/graduate-essay/tmp/datagrab/_run_2_subject_id_08/_fwhm_4/smooth".
230101-16:49:38,775 nipype.workflow INFO:
	 [Node] Executing "smooth" <nipype.interfaces.spm.preprocess.Smooth>


stty: 'standard input': Inappropriate ioctl for device


230101-16:50:05,309 nipype.workflow INFO:
	 [Node] Finished "smooth", elapsed time 26.533389s.
230101-16:50:05,311 nipype.workflow INFO:
	 [Node] Setting-up "datagrab.sinker" in "/home/chan/Documents/graduate-essay/tmp/datagrab/_run_1_subject_id_08/_fwhm_4/sinker".
230101-16:50:05,313 nipype.workflow INFO:
	 [Node] Outdated cache found for "datagrab.sinker".
230101-16:50:05,315 nipype.workflow INFO:
	 [Node] Executing "sinker" <nipype.interfaces.io.DataSink>
230101-16:50:05,317 nipype.workflow INFO:
	 [Node] Finished "sinker", elapsed time 0.000454s.
230101-16:50:05,318 nipype.workflow INFO:
	 [Node] Setting-up "datagrab.sinker" in "/home/chan/Documents/graduate-essay/tmp/datagrab/_run_2_subject_id_08/_fwhm_4/sinker".
230101-16:50:05,321 nipype.workflow INFO:
	 [Node] Executing "sinker" <nipype.interfaces.io.DataSink>
230101-16:50:05,322 nipype.workflow INFO:
	 [Node] Finished "sinker", elapsed time 0.000363s.


stty: 'standard input': Inappropriate ioctl for device


In [29]:
workflow.write_graph(graph2use='flat')

230101-16:43:20,783 nipype.workflow INFO:
	 Generated workflow graph: /home/chan/Documents/graduate-essay/tmp/datagrab/graph.png (graph2use=flat, simple_form=True).


'/home/chan/Documents/graduate-essay/tmp/datagrab/graph.png'

In [1]:
from nipype.utils.filemanip import loadpkl

res = loadpkl('./multi_atlas/inv_nl_register/_fwhm_4_run_1_subject_id_08/fnirt/_inputs.pklz')
res

{'ref_file': '/home/chan/Documents/graduate-essay/fsl/std/MNI152_T1_2mm_brain.nii',
 'in_file': '/home/chan/Documents/graduate-essay/processed/smoothed/_run_1_subject_id_08/_fwhm_4/swarsub-08_task-flanker_run-1_bold.nii',
 'affine_file': '/home/chan/Documents/graduate-essay/multi_atlas/inv_nl_register/_fwhm_4_run_1_subject_id_08/flirt_aff/swarsub-08_task-flanker_run-1_bold_flirt.mat',
 'fieldcoeff_file': True,
 'warped_file': 'warped',
 'output_type': 'NIFTI_GZ',
 'environ': {'FSLOUTPUTTYPE': 'NIFTI_GZ'}}

In [ ]:
res = loadpkl('./multi_atlas/inv_nl_register/_fwhm_4_run_1_subject_id_08/fnirt/_node.pklz')
res